In [ ]:

!pip install langchain
!pip install pinecone-client
!pip install openai
!pip install tiktoken
!pip install langchain-pinecone

In [ ]:
pip install langchain


In [ ]:
!pip install --upgrade groq



In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, WebBaseLoader, YoutubeLoader, DirectoryLoader, TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sklearn.metrics.pairwise import cosine_similarity
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from pinecone import Pinecone
from openai import OpenAI
import numpy as np
import tiktoken
import os

In [ ]:
import os
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
import pinecone
import requests
from tqdm import tqdm

loader = DirectoryLoader(
    src_dir,
    glob="**/*.txt",  # Add glob pattern
    show_progress=True,
    loader_cls=TextLoader,
    silent_errors=True  # Add error handling
)

# Replace OpenAI embeddings with Groq model API calls
def groq_embeddings(texts, groq_api_key):
    """Call Groq API to generate embeddings for the provided texts."""
    url = "https://api.groq.com/v1/embeddings"  # Replace with Groq's actual endpoint for embeddings
    headers = {
        "Authorization": f"Bearer {groq_api_key}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "mixtral-8x7b-32768",  # Specify the Groq model
        "texts": texts
    }

    response = requests.post(url, json=payload, headers=headers)

    if response.status_code == 200:
        embeddings = response.json()['embeddings']
        return embeddings
    else:
        print(f"Error: {response.status_code}")
        return []

def convert_files_to_txt(src_dir, dst_dir):
    """Convert repository files to txt format."""
    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)

    for root, dirs, files in os.walk(src_dir):
        for file in files:
            file_path = os.path.join(root, file)
            rel_path = os.path.relpath(file_path, src_dir)

            new_root = os.path.join(dst_dir, os.path.dirname(rel_path))
            os.makedirs(new_root, exist_ok=True)

            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = f.read()
            except UnicodeDecodeError:
                try:
                    with open(file_path, 'r', encoding='latin-1') as f:
                        data = f.read()
                except UnicodeDecodeError:
                    print(f"Failed to decode the file: {file_path}")
                    continue

            new_file_path = os.path.join(new_root, file + '.txt')
            with open(new_file_path, 'w', encoding='utf-8') as f:
                f.write(data)

PINECONE_API_KEY = "pcsk_5SLRZU_8hQy5wqViDGbcQbUrRa9d2Qg5Jy5D5oWhbMqxBFrpSzS6C9YEn8ec6McCEkLQ6y"
def ingest_files(src_dir, groq_api_key, pinecone_index_name):
    """Process and ingest files into Pinecone."""
    # Load documents
    loader = DirectoryLoader(
        src_dir,
        glob="**/*.txt",
        show_progress=True,
        loader_cls=TextLoader,
        silent_errors=True
    )
    repo_files = loader.load()
    print(f"Number of files loaded: {len(repo_files)}")

    # Split text into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100
    )
    documents = text_splitter.split_documents(documents=repo_files)
    print(f"Number of documents: {len(documents)}")

    # Clean up file paths
    for doc in documents:
        old_path = doc.metadata["source"]
        new_path = old_path.replace(".txt", "")
        doc.metadata.update({"source": new_path})

    # Generate embeddings using Groq
    texts = [doc.page_content for doc in documents]
    embeddings = groq_embeddings(texts, groq_api_key)

    # Initialize Pinecone client
    from langchain_pinecone import Pinecone

    # Store in Pinecone
    print(f"Inserting {len(documents)} documents into Pinecone")
    vectorstore = Pinecone.from_documents(
        documents=documents,
        embedding=embeddings,
        index_name=pinecone_index_name
    )
    print("Successfully inserted documents into Pinecone")



def main():
    # Configuration
    REPO_URL = "https://github.com/rakeshpuppala2590/Brain_Sight"
    GROQ_API_KEY = "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"  # Add your Groq API key here
      # Add your Pinecone API key here
    PINECONE_ENVIRONMENT = "us-east-1"
    PINECONE_INDEX_NAME = "reviewagent"

    # Clone repository
    print("Cloning repository...")
    os.system(f"git clone {REPO_URL} codebase")

    # Convert files to txt
    print("Converting files to txt format...")
    convert_files_to_txt("codebase", "converted_codebase")

    # Process and ingest files
    print("Processing and ingesting files...")
    ingest_files("converted_codebase", GROQ_API_KEY, PINECONE_INDEX_NAME)

    # Cleanup
    print("Cleaning up temporary files...")
    os.system("rm -rf codebase converted_codebase")

    print("Process completed successfully!")

if __name__ == "__main__":
    main()

NameError: name 'src_dir' is not defined